In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import cv2 as cv2
from  PIL import Image
from PIL import ImageFile 
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
from sklearn import metrics
import torch.nn as nn
package_paths = [
    '../input/timmmaster',
]

import sys


for pth in package_paths:
    sys.path.append(pth)
import timm
from torch.optim.lr_scheduler import OneCycleLR
from albumentations.core.composition import Compose,OneOf
from albumentations.pytorch import ToTensorV2

In [ ]:
class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b3_ns'
    pretrained = True
    img_size = 512
    num_classes = 100
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.2
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    num_epochs = 40
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
timm.list_models()[:10]

In [ ]:
df=pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")
img_path="../input/sorghum-id-fgvc-9/train_images"

In [ ]:
labels=df.cultivar.unique()
label_map={i:0 for i in labels}
for i,label in zip(range(100),labels):
    
    label_map[label]=i
df['cultivar']=df['cultivar'].map(label_map)    

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
df['fold']=-1
df.sample(frac=1).reset_index(drop=True)
for fold,(tr,val) in enumerate(kf.split(X=df,y=df.cultivar.values)):
    df.loc[val,"fold"]=int(fold)

In [ ]:
df.head()

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

# Display images for better understanding

In [ ]:
image=plt.imread(img_path+f"/{df.image.values[0]}")
plt.figure(figsize=(7,7))
plt.imshow(image,cmap='gray',vmin=0,vmax=255)

In [ ]:
class SorghumDataset:
    def __init__(self,df,aug):
        self.img_id=df.image.values
        
        self.img_id=    [i for i in self.img_id if str(i).split('.')[-1] =="png"]
                
        self.target=df.cultivar.values
        self.aug=aug
        #self.resize=resize
    def __len__(self):
        return len(self.img_id)
        
        
    def __getitem__(self,index):
        path="../input/sorghum-id-fgvc-9/train_images"
        
        image_path= path+"/"+self.img_id[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        target=self.target[index]
            
            
      
        if self.aug is not None:
            image=self.aug(image=image)['image']
            #image=self.aug(image)
#             image=np.transpose(image,(2,0,1)).astype(np.float32)
        
        return  {
            "image":image,
            "target":torch.tensor(target,dtype=torch.long)
        }
        
        
        
        
        
        
        
        

Train Function

In [ ]:
def cal_acc(preds,target):
    _,final_output=torch.max(preds,1)
    corr=torch.sum(final_output==target)
    return (corr*100)/len(preds)
        

In [ ]:
from tqdm import tqdm
def train(data_loader,model,optimizer,device,epoch):
    model.train()
    loop=tqdm(data_loader,leave=False)
    train_acc=[]
    total_acc=[]
    for data in loop:
        image=data["image"].to(device,dtype=torch.float)
        target=data["target"].to(device,dtype=torch.long)
        optimizer.zero_grad()
        output=model(image)
        loss=nn.CrossEntropyLoss()(output,target)
        acc=cal_acc(output,target)
        train_acc.append(acc)
        loop.set_description(f"Epoch->>>{epoch}")
        loop.set_postfix(loss=loss.item(),acc=acc)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    return sum(train_acc)/len(train_acc)

Evaluation Function

In [ ]:
def eval(data_loader,model,device):
    
    model.eval()
    final_target=[]
    final_output=[]
    
    with torch.no_grad():
        
        for data in tqdm(data_loader):
        
            image=data["image"].to(device,dtype=torch.float)
            target=data["target"].to(device,dtype=torch.long)
        
            output=model(image)
            
            target= target.detach().cpu().numpy().tolist()
            output= output.detach().cpu().numpy().tolist()
            final_target.extend(target)
            final_output.extend(output)
    return torch.tensor(final_target),torch.tensor(final_output)       
            

# Model

In [ ]:
import torch.nn as nn
import torchvision.models as models

def get_model():
    model=timm.create_model('tf_efficientnet_b3_ns', pretrained=True)
    #model=timm.create_model('resnet18', pretrained=True)
    in_features=model.get_classifier().in_features
    #in_features=model.fc.in_features
#     for param in model.parameters():
#         param.requires_grad=False
    model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, 100)
        )
#     model.fc =  nn.Linear(in_features, 100)
            
           
        
    return model
    
    

In [ ]:
X_train,X_valid,y_train,y_valid=sklearn.model_selection.train_test_split(df,df.cultivar.values,test_size=0.2)

In [ ]:
import albumentations as A
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ], p=0.5),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
                A.GridDropout(ratio=0.5, p=0.2),
                A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
            ], p=0.2),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# from torchvision import transforms
# mean=(0.485,0.456,0.406)
# std=(0.229,0.224,0.225)
# #aug=albumentations.Compose([albumentations.Normalize(mean,std,max_pixel_value=255.0,always_apply=True)])
# aug= albumentations.Compose([
#     albumentations.RandomResizedCrop(512,512),
#      albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     ToTensorV2(),
   
# ])

train_dataset= SorghumDataset(X_train,aug=get_transform('train'))
valid_dataset= SorghumDataset(X_valid,aug=get_transform('valid'))

In [ ]:

train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=16)
valid_loader=torch.utils.data.DataLoader(valid_dataset,batch_size=16)

In [ ]:
model=get_model()
device="cuda"
model.to(device)
epochs=10
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)
scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer, epochs=epochs, steps_per_epoch=len(train_loader),max_lr=1e-3, pct_start=0.2, 
                                                             div_factor=1.0e+3, final_div_factor=1.0+3)
best_accuracy=0
val_acc_list=[]
train_acc_list=[]
for epoch in range(epochs):
    train_acc=train(train_loader,model,optimizer,device,epoch)
    final_target,final_output=eval(valid_loader,model,device)
    _,final_output=torch.max(torch.nn.functional.softmax(final_output,1),1)
    cor=torch.sum(final_target==final_output)
    acc=(cor*100)/len(final_target)
    val_acc_list.append(acc.item())
    train_acc_list.append(train_acc.item())
    if acc>best_accuracy:
        torch.save(model.state_dict(),f"Epoch-> {epoch}md.pt")
        
    print((cor*100)/len(final_target))
    

In [ ]:
print(train_acc_list)
print(val_acc_list)